In [2]:
import sklearn
import numpy as np
import pandas as pd
import datetime
import bqplot.pyplot
import ipywidgets as ipy
import matplotlib.pyplot as plt


In [3]:
df=pd.read_csv("tweets_clean_data.csv")
df

,Unnamed: 0,userid,username,location,following,followers,totaltweets,usercreatedts,tweetid,retweetcount,hashtags,language,coordinates
0,0,1400378624998445057,RealBitstarz,"Phoenix, AZ",169,16,1573,2021-06-03 09:08:12.000000,1508957175405817862,0,"[{'text': 'FREESPINS', 'indices': [0, 10]}, {'...",en,NaN
1,1,2722503240,vubblePOP,Everywhere,2176,3176,62609,2014-08-10 21:38:54.000000,1508957175477059590,0,"[{'text': 'Russia', 'indices': [133, 140]}, {'...",en,NaN
2,2,128366364,am_queretaro,"Querétaro, México.",287,179968,231246,2010-03-31 22:46:52.000000,1508957175724457986,0,"[{'text': 'Ucrania', 'indices': [63, 71]}]",es,NaN
3,3,1439945731129987073,wtxnewsusa,New York,44,140,3470,2021-09-20 13:33:25.000000,1508957175833538564,0,"[{'text': 'usanews', 'indices': [61, 69]}, {'t...",en,NaN
4,4,1250894403448823812,Jclami1,NaN,290,19,2957,2020-04-16 21:10:56.000000,1508957176227844110,164,"[{'text': 'Ucrania', 'indices': [55, 63]}, {'t...",es,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
348859,348859,1500266680093528067,ObserverMiron,NaN,61,9,1043,2022-03-06 00:27:16.000000,1509319550705274884,776,[],en,NaN
348860,348860,431564396,TRUMP_TheDoodle,"Miami, USA",68,154,26385,2011-12-08 12:33:22.000000,1509319552471076876,631,"[{'text': 'Mariupol', 'indices': [92, 101]}]",en,NaN
348861,348861,1445137100282777761,Sam74243610,NaN,872,462,67344,2021-10-04 21:22:04.000000,1509319553876217857,3,[],en,NaN
348862,348862,41091628,Spaciousbeing,"iPhone: 0.000000,0.000000",1717,433,3823,2009-05-19 10:34:24.000000,1509319559345745925,6,"[{'text': 'Ukraine', 'indices': [126, 134]}]",en,NaN


In [14]:
df['username'].str.contains('ukraine', na=False)
df['username'].str.contains('Ukraine', na=False)
df['username'].str.contains('UKRAINE', na=False)



0         False
1         False
2         False
3         False
4         False
          ...  
348859    False
348860    False
348861    False
348862    False
348863    False
Name: username, Length: 348864, dtype: bool

In [16]:
ukraine_tweets= df.loc[(df['username'].str.contains('ukraine', na=False))| (df['username'].str.contains('ukraine', na=False))|(df['username'].str.contains('ukraine', na=False)) ]

In [23]:
list_hashtag=ukraine_tweets['hashtags'].tolist()

In [40]:
def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct

b=(Convert(list_hashtag))

In [68]:
df1=pd.read_csv('russia_losses_equipment.csv')
df2=pd.read_csv('russia_losses_personnel.csv')
eq=df1[[ 'aircraft', 'helicopter', 'tank', 'APC',
       'field artillery', 'MRL', 'military auto', 'fuel tank', 'drone',
       'naval ship', 'anti-aircraft warfare', 'special equipment',
       'mobile SRBM system']]
df1["Total Losses"] = eq.sum(axis=1)

In [73]:
#ax = fig.add_axes([0,0,1,1])
fig = plt.figure(figsize = (10, 5))

import plotly.express as px
cols = [ 'aircraft', 'helicopter', 'tank', 'APC',
       'field artillery', 'MRL', 'military auto', 'fuel tank', 'drone',
       'naval ship', 'anti-aircraft warfare', 'special equipment',
       'mobile SRBM system']
fig = px.line(df1, x = "date", y = cols,title="Equipment Loss vs Date")


fig.update_xaxes(rangeslider_visible=True)
fig.show()

<Figure size 720x360 with 0 Axes>

In [76]:
fig = px.bar(df2, x='day', y='personnel',color="personnel",title="Personnel Loss vs Days")
fig.show()